
<div  style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://www.sqlshack.com/wp-content/uploads/2020/07/anatomy-of-a-sql-table-1.png" alt="Databricks Learning" style="width: 600">
</div>

In [0]:
%python
files = dbutils.fs.ls("dbfs:/databricks-datasets/wikipedia-datasets/data-001/clickstream/raw-uncompressed-json/")
display(files)


## Parsing JSON Data

Mira los valores del json en un campo tipo STRING

In [0]:
SELECT `value` FROM json.`dbfs:/databricks-datasets/iot-stream/data-device`

Crea una tabla consultando los archivos json

In [0]:
CREATE OR REPLACE TABLE customers
AS
SELECT `value` FROM json.`dbfs:/databricks-datasets/iot-stream/data-device`

Mira la estructura y tipo de datos por campo

In [0]:
DESCRIBE customers

Parse cada campo de la cadena de JSON

In [0]:
SELECT `value`:user_id, `value`:calories_burnt, `value`:num_steps, `value`:miles_walked, `value`:time_stamp, `value`:device_id
FROM customers

Parsea el json usando la función from_json y definiendo el esquema de la cadena

In [0]:
SELECT from_json(`value`,"user_id INT, calories_burnt DOUBLE, num_steps INT, miles_walked DOUBLE, time_stamp TIMESTAMP, device_id STRING") AS profile_struct
  FROM customers;

Verifica los datos

In [0]:
SELECT `value`
FROM customers 
LIMIT 1

Crea una vista parseando los datos

In [0]:
CREATE OR REPLACE TEMP VIEW parsed_customers AS
  SELECT from_json(`value`, schema_of_json('{"user_id": 36, "calories_burnt": 163.90000915527344, "num_steps": 3278, "miles_walked": 1.6390000581741333, "time_stamp": "2018-07-20 07:34:28.546561", "device_id": 9}')) AS profile_struct
  FROM customers;
  
SELECT * FROM parsed_customers

Mira la estructura de la vista

In [0]:
DESCRIBE parsed_customers

Parsea el campo de la vista

In [0]:
SELECT  profile_struct.user_id, profile_struct.miles_walked
FROM parsed_customers

## Explode Function

In [0]:
%run ../Includes/GenData

Mira los datos y tipos de datos enfocándote en los campos con tipo de dato complejo (STRUCT , ARRAY)

In [0]:
SELECT *
FROM customer_shop

Usa la función explode para abrir los datos del vector

In [0]:
CREATE OR REPLACE TABLE customer_shops
AS
SELECT id, nombre, explode(compras) as compra
FROM customer_shop

## Collecting Rows

Compacta los valores del vector

In [0]:
SELECT nombre,
  collect_set(compra.producto) AS productos
FROM customer_shops
GROUP BY nombre


##Flatten Arrays

Telefonos repetidos con valores nulos

In [0]:
SELECT id,nombre, telefonos
FROM customer_shop

Telefonos unicos

In [0]:
SELECT id,nombre,array_distinct(telefonos) as telefonos
FROM customer_shop

Telefonos unicos y sin nulos

In [0]:
SELECT id,nombre,array_compact(array_distinct(telefonos)) as telefonos
FROM customer_shop

## Set Operations

In [0]:
%run ../Includes/GenData

Consulta la tabla con ordenes antiguas

In [0]:
SELECT * FROM old_orders

Consulta la tabla con las ordenes nuevas

In [0]:
SELECT * FROM new_orders

Utiliza la funcion UNION para concatenar los datos verticalmente eliminando los duplicados

In [0]:
SELECT * FROM old_orders 
UNION 
SELECT * FROM new_orders 

Utiliza la función UNION ALL para matener las duplicidades

In [0]:
SELECT * FROM old_orders 
UNION ALL
SELECT * FROM new_orders 

Obtiene los registros que están en ambas tablas

In [0]:
SELECT * FROM old_orders 
INTERSECT 
SELECT * FROM new_orders 

Obtienje los registros de la tabla superior que no estan en la tabla inferior

In [0]:
SELECT * FROM old_orders 
MINUS 
SELECT * FROM new_orders 

## Reshaping Data with Pivot

Consulta la tabla de ventas con datos por día de la semana

In [0]:
SELECT * FROM ventas

Con la función PIVOT puedes convertir los datos verticales en columnas

In [0]:


SELECT * FROM (
  SELECT
    producto,
    dia_semana,
    valor_venta
  FROM ventas
) PIVOT (
  sum(valor_venta) FOR dia_semana in (
    '1_LUN', '2_MAR', '3_MIE', '4_JUE', '5_VIE', '6_SAB', '7_DOM'
  )
);

Prueba la función pivot en Pyspark

In [0]:
%python
from pyspark.sql import functions as F
df=spark.table("ventas")
df = df.groupBy("producto").pivot("dia_semana").agg(
    F.sum("valor_venta").alias("valor_venta")
                               )
display(df)

Prueba la función contraria UNPIVOT

In [0]:
SELECT * FROM sales_month

Utiliza UNPIVOT con pyspark

In [0]:
%python
df=spark.table("sales_month")
df_unpivot = df.selectExpr(
    "region",
    "producto",
    "stack(3, "  # número de columnas a “desapilar”
    "'Enero', ventas_enero, "
    "'Febrero', ventas_febrero, "
    "'Marzo', ventas_marzo) as (mes, ventas)"
)
display(df_unpivot)

Utiliza UNPIVOT con SQL

In [0]:
SELECT region, producto, mes, ventas
FROM (
  SELECT region, producto,
         stack(3,
           'Enero', ventas_enero,
           'Febrero', ventas_febrero,
           'Marzo', ventas_marzo
         ) AS (mes, ventas)
  FROM sales_month
)